A partir de: https://github.com/Datawheel/py-economic-complexity/blob/main/docs/TUTORIAL.ipynb

que es un tuto pequeño, vamos a hacer un simulador para replicar los resultados de:

https://cnet.fi.uba.ar/netscix23/abstracts/Dynamics%20matter:%20A%20simulation%20framework%20to%20study%20diffusion%20processes%20on%20a%20Dynamic%20Product%20Space.pdf

como primera parte de nuestro análisis.

TODO:
además de terminar este collab, estaría bueno:
- separar en tres collabs, modelo, datos, informe
- traer de cada collab de alguna manera (grabando a un archivo tal vez?) las cosas

Partes de este collab.
- primero juego con los datos y veo que podamos usar la biblioteca encontrada
  - los args de las funciones toman el RCA, pero plot_twist, al menos las que vamos a usar también tomar el M, (la matriz RCA binarizada) así que zafamos
  - algunas funciones útiles
- luego viene:
- la definición de la interfaz de simulación
- una implementación naive para algo parecido al paper
- TODO la primera parte de nuestra idea de https://docutopia.sustrato.red/s/EIgp4w5cl#

In [ ]:
%pip install economic_complexity

from urllib.parse import urlencode

import economic_complexity as ecplx
import numpy as np
import pandas as pd
import time

In [ ]:
import requests

def encode_url(base_url, params):
  return f"{base_url}?{urlencode(params)}"

def request_data_from(url):
  r = requests.get(url)
  return pd.DataFrame(r.json()["data"])

In [ ]:
YOUR_TOKEN = "REPLACE_HERE" # increiblemente esto funciona; tal vez sean datos falsos?

In [ ]:
eurl = encode_url("https://oec.world/api/olap-proxy/data.jsonrecords",
                             {
                                 "Year": "2018,2019,2020",
                                 "cube": "trade_i_baci_a_92",
                                 "drilldowns": "Exporter Country,Year,HS4",
                                 "measures": "Trade Value",
                                 "token": YOUR_TOKEN
                             })
# vemos que la url encodeada sea idéntica a la que sabemos que funciona
expected = "https://oec.world/api/olap-proxy/data.jsonrecords?Year=2018%2C2019%2C2020&cube=trade_i_baci_a_92&drilldowns=Exporter+Country%2CYear%2CHS4&measures=Trade+Value&token=REPLACE_HERE"
assert eurl == expected, f"Expected:\n{expected}\ngot:\n{eurl}"

In [ ]:
df_trade = request_data_from(eurl)
display(df_trade)

In [ ]:
df_trade.dtypes

Filtros de datos, comparar con:

https://colab.research.google.com/drive/1H4ozsm6eH9HC13p4vYuCPenq9W5R6Meq#scrollTo=uUM93z4MFTqk

y fundamentar un poco más

igual lo importante de este collab es el simulador y el boilerplate para obtener los datos.

saqué la parte de población y la url de los datos es del [collab](https://colab.research.google.com/drive/1H4ozsm6eH9HC13p4vYuCPenq9W5R6Meq#scrollTo=f45lVamHVPX6)

In [ ]:
df = df_trade.copy()

# Products with more than $1.5B in global exports between 2016-2018
df_products = df.groupby('HS4 ID')['Trade Value'].sum().reset_index()
df_products = df_products[df_products['Trade Value'] > 3*500000000]
# Countries with more than $3B in global exports between 2016-2018
df_countries = df.groupby('Country ID')['Trade Value'].sum().reset_index()
df_countries = df_countries[df_countries['Trade Value'] > 3*1000000000]

df_filter  = df[
  (df['Country ID'].isin(df_countries['Country ID'])) &
  (df['HS4 ID'].isin(df_products['HS4 ID']))
]

In [ ]:
df_pivot = pd.pivot_table(df_filter, index=['Country ID'],
                                     columns=['HS4 ID'],
                                     values='Trade Value')\
             .reset_index()\
             .set_index('Country ID')\
             .dropna(axis=1, how="all")\
             .fillna(0)\
             .astype(float)

In [ ]:
df_pivot

Ahora calculo el RCA, ECI y PCI, como dice el tuto:

In [ ]:
rca = ecplx.rca(df_pivot)
ECI, PCI = ecplx.complexity(rca)

In [ ]:
type(ECI)

In [ ]:
# complejidad de los paises
ECI

In [ ]:
# complejidad del producto
PCI

In [ ]:
sorted_eci=ECI.sort_values(ascending=False)
type(sorted_eci)

In [ ]:
from pandas import DataFrame

def get_country_name(country_id, df: DataFrame):
  return df[df["Country ID"]==country_id]["Country"].values[0]

In [ ]:
# el más complejo!
get_country_name(sorted_eci.index[0], df)

In [ ]:
# el menos complejo... jajajaajajaaja CHAD
get_country_name(sorted_eci.index[-1], df)

Y el proximity? ... veamos que hay

In [ ]:
proximity = ecplx.proximity(rca)
proximity

In [ ]:
type(proximity)

In [ ]:
# si quiero los productos
proximity.columns

In [ ]:
proximity.index

Ok, esto está muy bien, pero vamos a tener que recalcular esto iteración a iteración y que estas funciones reciban los valores de comercio en dinero no ayuda nada; no se porque estos métodos no tomar como parámetro la matriz de resumen M, binarizada... creo que deberían poder, si nos fijamos en:

https://github.com/Datawheel/py-economic-complexity/blob/70abdb07d3651d710631f113aaa1ed4a665cdf7d/economic_complexity/product_space.py#L9

https://github.com/Datawheel/py-economic-complexity/blob/70abdb07d3651d710631f113aaa1ed4a665cdf7d/economic_complexity/complexity.py#L16

lo primero que hacen es binarizar con:

In [ ]:
M = rca.ge(1.0).astype(int)
M

In [ ]:
type(M)

In [ ]:
M.dtypes

In [ ]:
type(M.index[0])

In [ ]:
M.index

In [ ]:
M.index # indice paises

In [ ]:
M.columns # indice productos

In [ ]:
M.loc['afago'] # productos exportados por afago (tienen 1)

In [ ]:
M.loc['afago'][10101]

y en ningunos de los métodos citados vuelve a usar el argumento, así que creo que se debería cumplir que usando la matriz M:

In [ ]:
ECI_m, PCI_m = ecplx.complexity(M)
proximity_m = ecplx.proximity(M)

de lo mismo que usar los datos originales:

In [ ]:
assert ECI.equals(ECI_m)
assert PCI.equals(PCI_m)
assert proximity.equals(proximity_m)

**excelente! lo podemos usar.**

In [ ]:
proximity.index

Esto establece un grafo de productos, lo que tenemos que hacer es modelar un país/agente.

El mismo tendrá acceso a la matriz de exportación y será reponsabilidad de cada agente modificarla.

Luego un simulador se encargará del book keeping de simulación, modificación de estado compartido que no sea M, etc

In [ ]:
 %pip install pandera # para type checking, me parece que es super obviable

In [ ]:
# esto me hace acordar a primegean seetheando con "no resuelvas problemas que todavía no tienes"
# el nivel de abstracción de cabeza al dope que me mandé... pero ... parece que a la IA le gusta
# porque me autocompleta todo de maravilla

from typing import List, Callable, Dict # , Self # recién en py3.11
from __future__ import annotations # para poder type hint la clase
import pandera
from pandera.typing import DataFrame, Series

HS4_Product_Id = int
Country_Id = str
Tiempo = int

class SubclassResponsability(Exception):
  def __init__(self):
    super().__init__("Subclass Responsability: este método tiene que ser implementado por una subclase")

class IPais:
  """Vamos por un diseño bien de objetos; aunque creo que si esto se expresa
  como multiplicación de matrices vuela, pero no vuela en mi mente no matemática
  ¿quién programa en APL?.

  Distintos Paises pueden tener
  - distintas estrategias de elección de productos.
  - distintas fronteras de productos.
  - distintos tiempos para alcanzar productos
  """

  def elegir_productos(self) -> List[HS4_Product_Id]:
    """Cada país tiene una forma de elegir los siguientes productos a ser producidos,
    es una lista porque podría haber más de uno en un solo turno
    es la difusión efectiva"""
    raise SubclassResponsability

  def tiempo_para_ser_competitivo(self, pid: HS4_Product_Id) -> Tiempo:
    "Cuanto tiempo hasta ser competitivo? no cambia el estado"
    raise SubclassResponsability

  def investigar_producto(self, pid: HS4_Product_Id) -> Tiempo:
    """Usamos la metáfora de investigar, devuelve el tiempo necesario para terminar de lograrlo,
    dado el estado del agente"""
    raise SubclassResponsability

  def frontera_de_productos(self) -> List[HS4_Product_Id]:
    "todos los productos alcanzables"
    raise SubclassResponsability

  def frontera_de_productos_df(self) -> DataFrame[HS4_Product_Id]:
    "todos los productos alcanzables"
    raise SubclassResponsability

  def es_exportado(self, pid: HS4_Product_Id) -> bool:
    raise SubclassResponsability

  def productos_exportados(self) -> List[HS4_Product_Id]:
    "los productos que ya se exportan"
    raise SubclassResponsability

  def investigando(self) -> List[HS4_Product_Id]:
    "Devuelve los productos actualmente bajo investigación"
    raise SubclassResponsability

  def actualizar_exportaciones(self, pdis: List[HS4_Product_Id]) -> IPais:
    """Modifica las exportaciones, según el estado actual del agente"""
    raise SubclassResponsability

  def conocer_estado_del_mundo(self, **estado_dict):
    "permite settear estados compartidos adicionales"
    raise SubclassResponsability

  def productos_en_investigacion(self) -> List[HS4_Product_Id]:
    raise SubclassResponsability

  def avanzar_tiempo(self) -> List[HS4_Product_Id]:
    "Avanza el tiempo y devuelve la lista de productos terminados en este turno"
    raise SubclassResponsability


def test_interface(pais: Pais):
  "estaría bueno testear la interfaz, tengo en un proyecto código que hace eso, después lo subo"
  pass

Cada agente debe implementar la interfaz IPais, para que el simulador pueda usarlo. (Hay algunos métodos de más)

Pero también hay pequeños objetos que implementan una fracción parcial de la misma, de alguna forma dada y reutilizable usando multiple herencia, se llaman Mixin. Son un patrón un poco polémico, porque vendrían a ser una poor man composition y la herencia múltiple suele traer problemas cuando el grafo de herencia crece mucho, pero para proyectos chicos a mi me agrada bastante:

In [ ]:
class PaisBaseMixin:
  """se encarga de la investigación y la exportación, esto es común para cualquier pais
  En este contexto cada país se encarga de actualizar
  un estado compartido: la matriz M de exportaciones competitivas, del resto se encarga
  el Simulador.
  Este es un comportamiento base, sobre el cual cada instancia de Pais puede desarrollar"""
  def __init__(self, country_id: Country_Id, M: DataFrame[bool]):
    self.M = M
    self.country_id = country_id
    self.country_name = get_country_name(country_id, df) # TODO, no depender de una var global
    self.investigando: Dict[HS4_Product_Id, Tiempo] = {}
    #start = time.time()
    #print(f"(PaisBaseMixin.__init__) productos exportados calculados en: {time.time() - start}")

  def investigando(self) -> List[HS4_Product_Id]:
    return list(self.investigando.keys())

  def investigar_producto(self, pid: HS4_Product_Id) -> Tiempo:
    tiempo = self.tiempo_para_ser_competitivo(pid)
    self.investigando[pid] = tiempo
    return tiempo

  def productos_en_investigacion(self) -> List[HS4_Product_Id]:
    return list(self.investigando.keys())

  def avanzar_tiempo(self) -> List[HS4_Product_Id]:
    terminados = []
    for pid, tiempo in self.investigando.items():
      if tiempo == 1:
        terminados.append(pid)
      else:
        self.investigando[pid] -= 1
    for pid in terminados:
      del self.investigando[pid]
    return terminados

  def actualizar_exportaciones(self, pids: List[HS4_Product_Id]) -> IPais:
    """Modifica las exportaciones."""
    for pid in pids:
      self.M.loc[self.country_id][pid] = 1
    return self

  def productos_exportados_df(self) -> Index[HS4_Product_Id]:
    productos_pais = self.M.loc[self.country_id]
    return productos_pais[productos_pais == 1].index

  def productos_exportados(self) -> List[HS4_Product_Id]:
    return self.productos_exportados_df().to_list()

  def es_exportado(self, pid: HS4_Product_Id) -> bool:
    return self.M.loc[self.country_id][pid] == 1

  def __str__(self):
    return self.country_name

In [ ]:
class PaisConCotaProximidadMixin:
  """Por ejemplo, un Pais en un mundo simple o inocente
  logra siempre lo que quiere, en cada paso de la simulación, siempre y cuando
  el producto esté a su alcance"""

  def __init__(self,
               M: DataFrame[int],
               proximity: DataFrame[float],
               omega: int):
    self.proximity = proximity
    self.omega = omega

  def frontera_de_productos_df(self) -> DataFrame[HS4_Product_Id]:
    "Todos los productos alcanzables, devuelve una máscara"
    productos_pais = self.M.loc[self.country_id]
    exportados = productos_pais[productos_pais == 1]
    no_exportados = productos_pais[productos_pais == 0]

    frontera = ( self.proximity.loc[exportados.index][no_exportados.index] > self.omega )
    frontera = frontera.any(axis='rows')
    return frontera[frontera].index
    #frontera = frontera[frontera].index
    #return frontera

  def frontera_de_productos(self) -> List[HS4_Product_Id]:
    return self.frontera_de_productos_df().to_list()

  def elegir_productos(self) -> List[HS4_Product_Id]:
    return self.frontera_de_productos()

  def conocer_estado_del_mundo(self, **kwargs):
    self.proximity = kwargs["proximidad"]


class PaisNaive(PaisBaseMixin, PaisConCotaProximidadMixin, IPais):
  """Y por ejemplo ahora podemos definir un pais simple como una combinación de mixins
  tener cuidado con el orden de los mixins... la interfaz es el último
  ¿performance? ¿quien quiere tal cosa si puede jugar con interfaces?

  Este pais logra en cada iteración accede a ser competitivo, en un turno a todos
  los productos que estén a su alcance.
  """
  def __init__(self, country_id: Country_Id,
               M: DataFrame, proximidad: DataFrame, omega: float):
    "el constructor aglutina todo"
    PaisBaseMixin.__init__(self, country_id, M)
    PaisConCotaProximidadMixin.__init__(self, M, proximidad, omega)

  def tiempo_para_ser_competitivo(self, pid: HS4_Product_Id) -> Tiempo:
    return 1



Un coordinador de simulación es un sistema que:
- toma datos e inicializa agentes
- itera hasta encontrar un criterio de parada.
  - En cada iteración:
    - le indica a cada agente que elija un producto a investigar
    - coordina cuando se actualiza el estado del mundo compartido

Una implementación del comportamiento del paper citado

In [ ]:
from typing import Iterator
import time

class Simulador:
  """framework de caja blanca ... HW cry me a river,
  en fin, el simulador ... ejem... simula, debería ser self explanatory"""

  def __init__(self,
               criterio_parada: Callable[..., bool]):
    self.criterio_parada = criterio_parada
    self._estado_inicial_de_parada()
    start = time.time()
    self.paises = self._crear_paises()
    print(f"paises creados en: {time.time() - start}")

  def simular(self):
    for _ in self.iterar_simulacion():
      pass

  # TODO: que devuelva lo que usa para contar el criterio de parada, un counter, indice gini, etc
  def iterar_simulacion(self) -> Iterator[Dict[Country_Id, HS4_Product_Id]]:
    """Devuelve un iterador para poder simularlo por pasos y obtener para cada país
    que productos alcanzaron competitividad"""
    while not self.es_fin_de_simulacion():
      output = {}
      # fase de decisiones    (nota, la tener dos fases es muy paralilizable)
      for pais in self.paises:
        nuevos_productos = pais.elegir_productos()
        for pid in nuevos_productos:
          pais.investigar_producto(pid)
      # fase de acciones
      for pais in self.paises:
        terminados = pais.avanzar_tiempo()
        output[pais] = terminados # como se podría generalizar lo que devuelve?
        pais.actualizar_exportaciones(terminados)

      self._actualizar_estado(output)
      self._notificar_paises()
      yield output

  def es_fin_de_simulacion(self) -> bool:
    raise SubclassResponsability

  # privadas/protegidas
  def _crear_paises(self):
    raise SubclassResponsability

  def _actualizar_estado(self, output_iteracion: Dict[Country_Id, HS4_Product_Id]):
    """se encarga del book keeping y la transiciones de estado que no sean,
    responsabilidad del país. Entre ellas, el conteo de pasos por ejemplo"""
    raise SubclassResponsability

  def _notificar_paises(self):
    """cuando el estado del mundo cambia los paises deben enterarse para
    actualizar de ser necesario algún estado interno adicional a las exportaciones"""
    raise SubclassResponsability

  def _estado_inicial_de_parada():
    raise SubclassResponsability


Con esto podemos definir de manera muy simple, los dos simuladores que usan en el paper:

In [ ]:
class SimuladorProductSpace(Simulador):
  """Simulador base, con paises Naive que opera sobre el grafo de proximidades,
  -mal llamado- product space"""

  def __init__(self,
               criterio_parada: Callable[..., bool],
               M: DataFrame, omega=0.4):
    self.M = M
    self.omega = omega
    start = time.time()
    self.proximidad = ecplx.proximity(M)
    print(f"proximidad calculada en: {time.time() - start}")
    # llamar al constructor de las super clases al final
    super().__init__(criterio_parada)

  # TODO esto debería ser un wrapper así puedo cambiar facilmente el tipo de país
  # y puedo sacar del constructor el omega, podría tener una simulación con toda esta lógica
  # pero que no use omega
  def _crear_paises(self):
    return [PaisNaive(country_id, self.M, self.proximidad, self.omega) for country_id in self.M.index]

  def _estado_inicial_de_parada(self):
    self.current_step = 0

  def _actualizar_estado(self, _):
    # este simulador no actualiza nada más
    self.current_step += 1

  def es_fin_de_simulacion(self):
    return self.criterio_parada(self.current_step)

In [ ]:
class SimuladorEstatico(SimuladorProductSpace):
  def _notificar_paises(self):
    pass # no hace falta hacer nada, porque no hay modificación

class SimuladorDinamico(SimuladorProductSpace):
  def _actualizar_estado(self, _):
    self.proximidad = ecplx.proximity(self.M)
    super()._actualizar_estado(_)

  def _notificar_paises(self):
    for p in self.paises:
      p.conocer_estado_del_mundo(proximidad = self.proximidad) # se podría pasar directamente por eficiencia, pero interfaz

In [ ]:
def print_m(msg, mostrar=True):
  if mostrar:
    print(msg)

def correr_simulacion_mostrando(sim: Simulador, mostrar = True) -> List[Dict[Country_Id, List[HS4_Product_Id]]]:
  "horrible esta función"
  res = []
  start = time.time()
  it_start = time.time()
  for d in sim.iterar_simulacion():
    res.append(d)
    print_m(f"iteración: {sim.current_step}", mostrar)
    for pais, productos in d.items():
      print_m(f"\t{pais.country_id}: descubrió {len(productos)}\t({pais})", mostrar)
    print_m(f"tiempo iteración: {time.time() - it_start}", mostrar)
    it_start = time.time()
  print_m(f"tiempo total: {time.time() - start}", mostrar)
  return res

In [ ]:
simEst = SimuladorEstatico(lambda step: step > 4, M.copy(deep=True), 0.4)
historia_estatica = correr_simulacion_mostrando(simEst, False)
historia_estatica

In [ ]:
simDin = SimuladorDinamico(lambda step: step > 4, M.copy(deep=True), 0.4)
historia_dinamica = correr_simulacion_mostrando(simDin)

In [ ]:
simDin_hard = SimuladorDinamico(lambda step: step > 4, M.copy(deep=True), 0.4)
historia_dinamica_hard = correr_simulacion_mostrando(simDin_hard)

In [ ]:
# TODO, criterio de parada térmico (que frene cuando los cambios sean pocos)
import math

class PaisComplejo(PaisNaive, IPais):
  """Ok, este es el problema de los frameworks de subclasificación,
  esto no es un pais Naive, es todo lo contrario, sin embargo subclasifica
  porque es cómodo por el código compartido"""
  def __init__(self, country_id: Country_Id,
               M: DataFrame,
               proximidad: DataFrame,
               eci, PCI,
               omega: float):
    PaisNaive.__init__(self, country_id, M, proximidad, omega)
    self.mi_eci = eci
    self.PCI = PCI

  def tiempo_para_ser_competitivo(self, pid: HS4_Product_Id) -> Tiempo:
    """TODO: discutir esto. además no se porque ECI y PCI dan negativos (ver más arriba)
    esto es muy similar al cálculo de proximidad, pero en vez del máximo como define el paper se calcula el mínimo
    (nosotros no calculamos el máximo, porque como es un umbral usamos .any(), ver el método frontera_de_productos_df)
    """
    complejidad = self.PCI[pid]
    return math.ceil(abs((complejidad / self.mi_eci)))

  def tiempos_para_ser_competitivo(self) -> Series[Tiempo]:
    frontera = self.frontera_de_productos_df()
    tiempos = frontera.map(self.tiempo_para_ser_competitivo)
    return pd.Series(index=frontera, data=tiempos)

class PaisHormiga(PaisComplejo):
  def elegir_productos(self) -> List[HS4_Product_Id]:
    #TODO: por qué el 10% y no 1 o config? ... я не знаю, потому
    "selecciona los primeros 10% de productos más complejos de su frontera"
    tiempos = self.tiempos_para_ser_competitivo()
    return tiempos.nlargest(int(len(tiempos) * 0.1)).index.to_list()

class PaisCigarra(PaisComplejo):
  def elegir_productos(self) -> List[HS4_Product_Id]:
    "selecciona los últimos 10% de productos más complejos de su frontera"
    tiempos = self.tiempos_para_ser_competitivo()
    return tiempos.nsmallest(int(len(tiempos) * 0.1)).index.to_list()

class SimuladorComplejo(SimuladorProductSpace):
  "Un simulador complejo tiene paises que consideran su complejidad"
  def __init__(self,
               criterio_parada: Callable[..., bool],
               M: DataFrame, omega=0.4):
    self.ECI, self.PCI = ecplx.complexity(M)
    super().__init__(criterio_parada, M, omega)

  def _actualizar_estado(self, output_iteracion: Dict[Country_Id, HS4_Product_Id]):
    self.ECI, self.PCI = ecplx.complexity(M)
    super()._actualizar_estado(output_iteracion)

  def _notificar_paises(self):
    for p in self.paises:
      p.conocer_estado_del_mundo(proximidad = self.proximidad,
                                 eci=ECI[p.country_id], PCI=PCI)

class SimuladorCigarras(SimuladorComplejo):
  def _crear_paises(self):
    return [PaisCigarra(country_id, self.M,
                         self.proximidad,
                         self.ECI[country_id],
                         self.PCI,
                         self.omega) for country_id in self.M.index]

class SimuladorHormigas(SimuladorComplejo):
  def _crear_paises(self):
    return [PaisHormiga(country_id, self.M,
                         self.proximidad,
                         self.ECI[country_id],
                         self.PCI,
                         self.omega) for country_id in self.M.index]

In [ ]:
#TODO el omega tiene que salir de los resultados anteriores
simDin_cigarra = SimuladorCigarras(lambda step: step > 4, M.copy(deep=True), 0.4)
correr_simulacion_mostrando(simDin_cigarra)

In [ ]:
simDin_hormiga = SimuladorHormigas(lambda step: step > 4, M.copy(deep=True), 0.4)
correr_simulacion_mostrando(simDin_hormiga)